# SQL Databases on hadoop

Relationele Database Management Systemen zijn traditioneel verticaal schaalbaar.
Dit conflicteert met Big Data toepassingen waar we de voorkeur geven aan horizontale schaalbaarheid om onder andere de kosten te verlagen, de beschikbare rekenkracht of performantie en fout-tolerantie te verbeteren.
Hierdoor kunnen de reeds gekende RDBMS niet gebruikt worden.

De volgende populaire applicaties kunnen gebruikt worden om SQL te gebruiken op hadoop:
* Apache Hive
* Cloudera Impala
* Presto
* Shark

Hierna gaan we een kort overzicht geven van al deze applicaties en daarna focussen we op een toepassingen van Apache Hive.

## Apache Hive

Deze applicatie is ontwikkeld door Facebook als **datawarehouse framework** voor interne toepassingen en is snel zeer populair geworden om queries uit te voeren op Hadoop.
Het is belangrijk om in gedachten te houden dat hoewel Hive een **SQL-like querying omgeving** aanbiedt, dat er in de achtergrond een **MapReduce methodologie** gebruikt wordt om de database te bevragen.
Dit wil zeggen dat de **queries gecompileerd moeten worden naar MapReduce toepassingen**.
Hive ondersteund ook gebruikers-gedefineerde functies en laat toe om gecomprimeerde data te verwerken.
Momenteel wordt Hive verder verbeterd en uitgebreid door HortonWorks (Cloudera) dat een nieuwe backend aan het uitwerken is (Tez Project) om de responstijd van Hive te verbeteren.

Voordelen:
* Op bijna alle Hadoop installaties standaard geinstalleerd
* Goede tool om te proberen door minimale start-investering (gratis)

Nadelen:
* Niet meest snelle manier door overhead van MapReduce (batch processing)
* Enkel 4 file-formats ondersteund:
 * Text, SequenceFile, ORC, RCFile

## Cloudera Impala

Maakt het mogelijk om interactieve SQL queries uit te voeren op HDFS en HBase. 
Impala voert **queries uit in real time** en verbeterd daardoor de performantie  door geen batch processing te gebruiken.
Daarnaast wordt ook het **gebruik van verscheidene SQL-based bedrijfsanalyse tools mogelijk** gemaakt.
Deze applicatie is een open source applicatie ontwikkeld door Cloudera.

Voordelen:
* Sneller dan Hive
* Ondersteund cloud based architecture door Amazon's Elastic MapReduce
* Is compatibel met ANSI SQL (standaard SQL standaard)
* Integratie met business intelligence tools mogelijk

Nadelen:
* Moeilijker op te zetten
* Volledige kracht maar beschikbaar bij gebruik van Parquet file format
* Geen support voor YARN
* Vereist installatie van daemons op elke node

## Presto

Een tweede applicatie ontwikkeld door Facebook.
Ook deze applicatie is open source.
Deze applicatie is geschreven in Java en heeft een groot aantal kenmerken gemeen met Impala, bijvoorbeeld:
* Een interactieve ervaring
* Moeiljk om op te zetten (installatie op de verscheidene nodes)
* Vereist een specifiek file format voor data opslag (RCFile)

Daarnaast biedt Presto wel compatibiliteit met de Hive meta-store en laat Presto toe om data van verscheidene bronnen te combineren.
Het grootste verschil met Impala is dat Presto niet ondersteund wordt door veel leverancies van cloud-toepassingen, ook al maken reeds een aantal grote bedrijven er gebruik van (bijvoorbeeld AirBnb en Dropbox).

## Shark

Deze applicatie is ontstaan om een alternatief te bieden voor Hive met MapReduce.
Het doel was om alle functionaliteiten van Hive te behouden maar de performantie te verbeteren.
Deze tool is geschreven in Scala door UC Berkeley en zoals de naam doet vermoeden maakt het gebruik van **Spark**.
Tot op een zeker punt kan Shark de performantie van Hive verbeteren maar de **schaalbaarheid van de tool** is niet zo goed als Hive.
Dit komt omdat het **gebouwd is boven op Hive** waardoor het de complexe codebase van Hive heeft overgeerfd heeft.
Het onderhoud en aanpassen van deze codebase zonder in te boeten op performantie is echter niet eenvoudig.

## Spark SQL

Dit onderdeel van Spark biedt de mogelijkheid aan om Spark Queries uit te voeren op ingeladen Dataframes.
Omdat dit gebruik maakt van Spark biedt het veel voordelen en is de performantie beter dan tools die gebruik maken van MapReduce.
Het grootste nadeel is echter dat deze data niet standaard opgeslagen wordt op een harde schrijf.
Het is vrij eenvoudig om deze dataframes/tabellen op te slaan als bijvoorbeeld csv maar de relaties tussen kolommen van verschillende tabellen kan niet opgeslaan worden en vereist extra manueel werk om bij te houden.

## Voorbeeld toepassing: Hive

Om te beginnen moet Hive geinstalleerd worden.
Hiervoor kan je de stappen volgen op de site van hive.
Bij oudere versies kunnen er problemen opduiken door het gebruik van oudere java versies.
Om deze te vermijden kies je best voor voorgecompileerde versie van **apache hive versie 4 alpha 2**.

Plaats de extracte files in een folder met naam hive in je home folder.
Voeg dan aan de .bashrc file in je home folder onderaan het volgende toe:

* export HIVE_HOME=/home/bigdata/hive
* export PATH=$PATH:$HIVE_HOME/bin

Indien de volgende errors opkomen kan je de volgende zaken proberen:
* java.lang.NoSuchMethodError: 'void com.google.common.base.Preconditions.checkArgument
    * Verwijder guava-19.0.jar in hive/lib
    
Na het opnieuw inladen van de systeem variabelen (herstarten terminal of uitvoeren van het source .bashrc commando) kan je beginnen werken met Hive.

Eerst moet je wat initialisaties uitvoeren door middel van het volgende commando:

    schematool -dbType derby -initSchema

Nu kan je de Beeline CLI starten om te HQL queries uit te voeren op de Hive server met het volgende commando:

    $HIVE_HOME/bin/beeline -u jdbc:hive2://

Dit commando start twee zaken op, namelijk de hiveserver en de beeline CLI voor queries uit te voeren in 1 commando.
Dit kan gebruikt worden voor te testen en te experimenteren en niet voor in productie te werken.
De beeline CLI interfase is een sql-like console waarin we allerhande queries kunnen schrijven.
Een uitgebreide beschrijving van alle mogelijke queries vind je [hier](https://cwiki.apache.org/confluence/display/hive/languagemanual).
Bijvoorbeeld kunnen we de databases bekijken met

    show databases;
    
Maak ook een database aan waarin we we gaan werken deze les.
Schrijf hieronder de commando's die je nodig hebt om dit te doen.

In [ ]:
!create database odisee;
!use odisee;

Download nu deze zip files over [werknemers](https://github.com/RobinDong/hive-examples/blob/master/employee/employees.csv.gz) en [salarissen ](https://github.com/RobinDong/hive-examples/blob/master/employee/salaries.csv.gz).
Unzip daarna deze files en upload ze naar het hdfs.
Schrijf hieronder de nodige code om deze files in het hdfs op te slaan.

In [2]:
import pydoop.hdfs as hdfs

localFS = hdfs.hdfs(host='')
client = hdfs.hdfs(host='localhost', port=9000)

if not client.exists('/user/bigdata/SQL'):
    client.create_directory('/user/bigdata/SQL')

# do some cleaning in case anything is present
for f in client.list_directory("."):
    client.delete(f["name"], True)
        
# upload input.txt
localFS.copy("employees.csv", client, "SQL/employees/employees.csv")
localFS.copy("salaries.csv", client, "SQL/salaries/salaries.csv")

0

Deze files bevatten de data die we straks gaan inlezen in het Hive Datawarehouse. 
Hiervoor moeten we echter eerst de tabellen aanmaken waarin we deze data gaan opslaan.
Dit kan door middel van de volgende HQL commando's uit te voeren in de hive HQL.

Na het maken kunnen we de data inlezen door middel van het load data commando.

In [ ]:
create external table employee (
    employee_id INT,
    birthday DATE,
    first_name STRING,
    family_name STRING,
    gender CHAR(1),
    work_day DATE)
row format serde 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
stored as textfile;

In [ ]:
LOAD DATA INPATH '/user/bigdata/10_SQL/employees' overwrite into table employee;

In [ ]:
create external table salary (
    employee_id INT,
    salary INT,
    start_date DATE,
    end_date DATE)
row format serde 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
stored as textfile;

In [ ]:
LOAD DATA INPATH '/user/bigdata/10_SQL/salaries' overwrite into table salary;

Bestudeer nu opnieuw de [HQL language](https://cwiki.apache.org/confluence/display/Hive/LanguageManual) en stel de queries op die de volgende zaken opzoeken:
* De 10 oudste werknemers
* Het aantal werknemers dat gestart is in 1990
* De voor en familienaam en gemiddelde salaris van de 10 werknemers met het hoogste gemiddelde salaris. (Tip: gebruik order by ipv sort by om globale orde te bepalen in reducer)
* Is er een gender wage gap aanwezig in dit bedrijf? Bepaal hiervoor per geslacht het gemiddelde salaris aan de hand van een group by.

Buiten rechtstreeks te werken met de hive interface om rechtstreeks queries uit te voeren kunnen we dit ook met spark doen.
Hieronder staan de bovenstaande commando's omgezet naar queries met spark om het binnen een spark applicatie uit te voeren.